# Simulation xi_PC scores

In [1]:
from patsy import dmatrix
from scipy import stats
import numpy as np
import pandas as pd
import random
import time
import matplotlib.pyplot as plt
%matplotlib inline

np.set_printoptions(precision = 4)

In [2]:
random.seed(123)
n_repeat = 200

In [6]:
simu_n = 500
simu_sigmatrix = [[4.9, 0, 2.5, 1.4], 
                  [0, 3.6, 1, 1.2], 
                  [2.5, 1, 3, 0], 
                  [1.4, 1.2, 0, 2]]
simu_score = pd.DataFrame(np.random.multivariate_normal([0,0,0,0], simu_sigmatrix, simu_n))

# Simulation data X1measure

In [7]:
def fun_simu_X1_help(t):
    
    mu1 = 120 * (t - 0.5)**3 + 5 * (t - 0.5)**2 - 15 * t + 10
    psi11 = np.sqrt(2) * np.sin(2 * np.pi * t)
    psi12 = np.sqrt(2) * np.cos(2 * np.pi * t)
        
    return mu1, psi11, psi12

In [8]:
def fun_simu_X1measure(N, data_simu_score, lower, upper):
    
    pick = random.choices(range(lower, upper + 1), k = N)
    subject = [item for item, count in zip(range(1, N + 1), pick) for i in range(count)]
    
    xtime = np.array([])
    X1 = np.array([])
    for i in range(N):
        timepoint_i = np.append(0, np.sort(np.random.uniform(0, 1, pick[i] - 1)))
        xtime = np.append(xtime, timepoint_i)
        
        # generate X1
        mu1_i, psi11_i, psil2_i = fun_simu_X1_help(timepoint_i)
        X1_i = mu1_i + data_simu_score.iloc[i, 0] * psi11_i + data_simu_score.iloc[i, 1] * psil2_i
        X1 = np.append(X1, X1_i)
    
    W1 = X1 + np.random.normal(0, 1, len(X1))
        
    out = pd.DataFrame({'id': subject, 'xtime': xtime, 'X1': X1, 'W1': W1})

    return out

# Simulation data X2measure

In [9]:
def fun_simu_X2_help(t):
    
    mu2 = 12 * (t - 1)**4 + 15 * t**2 - 10
    psi21 = np.sqrt(2) * np.sin(4 * np.pi * t)
    psi22 = np.sqrt(2) * np.cos(4 * np.pi * t)
    
    return mu2, psi21, psi22

In [10]:
def fun_simu_X2measure(N, data_simu_score, lower, upper):
    
    pick = random.choices(range(lower, upper + 1), k = N)
    subject = [item for item, count in zip(range(1, N + 1), pick) for i in range(count)]
    
    xtime = np.array([])
    X2 = np.array([])
    for i in range(N):
        timepoint_i = np.append(0, np.sort(np.random.uniform(0.1, 1, pick[i] - 1)))
        xtime = np.append(xtime, timepoint_i)
        
        # generate X2
        mu2_i, psi21_i, psi22_i = fun_simu_X2_help(timepoint_i)
        X2_i = mu2_i + data_simu_score.iloc[i, 2] * psi21_i + data_simu_score.iloc[i, 3] * psi22_i
        X2 = np.append(X2, X2_i)
    
    W2= X2 + np.random.normal(0, np.sqrt(1.5), len(X2))
    
    out = pd.DataFrame({'id': subject, 'xtime': xtime, 'X2': X2, 'W2': W2})
    
    return out

In [11]:
simu_X2 = fun_simu_X2measure(200, data_simu_score = simu_score, lower=8, upper=10)

In [12]:
np.std(simu_X2[['W2']]) **2

W2    17.9205
dtype: float64

In [13]:
simu_X1 = fun_simu_X1measure(200, data_simu_score = simu_score, lower=8, upper=10)

In [14]:
np.std(simu_X1[['W1']]) **2

W1    19.954089
dtype: float64

# Simulation gamma matrix

In [15]:
simu_gamma = np.array([[0.2, 0.8, -1], [0.6, -1.3, -0.5]]).T

#simu_gamma = np.array([[0.7,-0.3,1.5]]).T

# Simulation beta

In [16]:
simu_beta = np.array([[3.18 , -1.79 ,2.52 ,1.48, -1.17],
[-0.42 ,2.68 ,-0.69 ,1.02, 0.54],
                     [-2.66 ,2.83 ,2.32 ,1.14, 0.76]]) 
simu_beta

array([[ 3.18, -1.79,  2.52,  1.48, -1.17],
       [-0.42,  2.68, -0.69,  1.02,  0.54],
       [-2.66,  2.83,  2.32,  1.14,  0.76]])

# Simulation data Ymeasure

In [17]:
def fun_simu_Ymeasure(N, data_simu_gamma, data_simu_beta, data_simu_score):
    
    ######################### Generate Z matrix function ########################
    simu_Z = np.vstack([np.repeat(1, N), 
                        np.random.binomial(n = 1, p = 0.5, size = N), 
                        np.random.normal(0, 1, N)]).T
    
    Y_error = np.random.normal(0, 2, N *5)
    
    # Check
    #print('simu_Z:', simu_Z)
    #print('The size of simu_Z:', simu_Z.shape)    
    
    ######################### Generate Cluster Label ############################
    n_cluster = np.shape(data_simu_gamma)[1] + 1
    
    temp_sub_prob = np.exp(np.dot(simu_Z, data_simu_gamma))
    sub_prob = temp_sub_prob / (1 + temp_sub_prob.sum(1, keepdims = True))
    prob = np.hstack((sub_prob, 1 - np.sum(sub_prob, axis = 1).reshape(-1, 1)))
    
    dummy_label = np.vstack(list(map(lambda i: np.random.multinomial(1, prob[i], size=1)[0], range(N))))
    simu_label = np.sum(dummy_label * range(1, n_cluster + 1), axis = 1)
    # Check
    #print('simu_label:', simu_label)
    #print('The size of simu_label:', simu_label.shape)
    
    ######################### Generate time point for Y #########################
    simu_Ytime = np.vstack([np.repeat(0, N)] + 
                           list(map(lambda mu: np.random.normal(mu, 0.05, N), 
                                    [0.3, 0.5, 0.7, 0.9]))).T
    # Check
    #print('simu_Ytime:', simu_Ytime)
    #print('The size of simu_Ytime:', simu_Ytime.shape)
    
    ######################### Generate mean part of Y ###########################
    Y_mu1, Y_psi11, Y_psil2 = fun_simu_X1_help(simu_Ytime)
    
    Y_mu2, Y_psi21, Y_psi22 = fun_simu_X2_help(simu_Ytime)
    
    Y_X1 = np.array([])
    Y_X2 = np.array([])
    for i in range(N):
        Y_X1_i = Y_mu1[i] + data_simu_score.iloc[i, 0] * Y_psi11[i] + data_simu_score.iloc[i, 1] * Y_psil2[i]
        Y_X1 = np.append(Y_X1, Y_X1_i) 
    
        Y_X2_i = Y_mu2[i] + data_simu_score.iloc[i, 2] * Y_psi21[i] + data_simu_score.iloc[i, 3] * Y_psi22[i]
        Y_X2 = np.append(Y_X2, Y_X2_i)
    # Check
    #print('Y_X1:', Y_X1)
    #print('The size of Y_X1:', Y_X1.shape)
    #print('Y_X2:', Y_X2)
    #print('The size of Y_X2:', Y_X2.shape)
    
    ######################### Generate Y value ##################################
    pd_temp = pd.DataFrame({'Label': np.repeat(simu_label, 5), 
                            'Intercept': 1, 
                            'X1': Y_X1,
                            'X2': Y_X2, 
                            'Z1': np.repeat(simu_Z[:,1], 5), 
                            'Z2': np.repeat(simu_Z[:,2], 5)})
    
    n1 = len(pd_temp[pd_temp.Label == 1])
    n2 = len(pd_temp[pd_temp.Label == 2])
    n3 = len(pd_temp[pd_temp.Label == 3])

    random1 = np.random.normal(0, np.sqrt(2), int(n1/5))

    intercept1 = np.repeat(random1, 5)

    random2 = np.random.normal(0, np.sqrt(1.5), int(n2/5))
    interpcet2 = np.repeat(random2, 5)
    
    random3 = np.random.normal(0, np.sqrt(1), int(n3/5))
    interpcet3 = np.repeat(random3, 5)

    Y1 = pd_temp[pd_temp.Label == 1].drop(['Label'], axis = 1).dot(data_simu_beta[0].reshape(-1, 1)) 
    Y1_random = Y1 + intercept1.reshape(-1, 1)    
    
    Y2 = pd_temp[pd_temp.Label == 2].drop(['Label'], axis = 1).dot(data_simu_beta[1].reshape(-1, 1)) 
    Y2_random = Y2 + interpcet2.reshape(-1, 1)

    Y3 = pd_temp[pd_temp.Label == 3].drop(['Label'], axis = 1).dot(data_simu_beta[2].reshape(-1, 1)) 
    Y3_random = Y3 + interpcet3.reshape(-1, 1)

    Y = pd.concat([Y1_random, Y2_random, Y3_random], axis=0).sort_index() + pd.DataFrame(Y_error)

    #############################################################################

    
    pd_temp['ID'] = np.repeat(range(1, N + 1), 5)
    pd_temp['Ytime'] = simu_Ytime.ravel()
    pd_temp['Y'] = Y
    
    simu_Ymeaure = pd_temp[['ID', 'Label', 'Ytime', 'Y', 'Z1', 'Z2', 'X1', 'X2']]
    
    return simu_Ymeaure, prob

In [18]:
simu_Ymeasure, simu_probability = fun_simu_Ymeasure(simu_n, simu_gamma, simu_beta, simu_score)


In [19]:
np.std(simu_Ymeasure[['Y']]) **2

Y    217.796038
dtype: float64

In [20]:
random.seed(123)
for i in range(n_repeat):
    
    simu_X1measure = fun_simu_X1measure(simu_n, simu_score, 8, 10)
    simu_X1measure.to_csv(path_or_buf = 'X1measure/X1measure_{}.csv'.format(i + 1))
    
    simu_X2measure = fun_simu_X2measure(simu_n, simu_score, 8, 10)
    simu_X2measure.to_csv(path_or_buf = 'X2measure/X2measure_{}.csv'.format(i + 1))
    
    simu_Ymeasure, simu_probability = fun_simu_Ymeasure(simu_n, simu_gamma, simu_beta, simu_score)
    simu_Ymeasure.to_csv(path_or_buf = 'Ymeasure/Ymeasure_{}.csv'.format(i + 1))
    pd.DataFrame(simu_probability).to_csv(path_or_buf = 'Probability/probability_{}.csv'.format(i + 1))

In [22]:
pd.DataFrame(simu_score).to_csv('true_score.csv')

In [23]:
pd.DataFrame(simu_beta).to_csv('true_beta.csv')